In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
import Graph_EEGresnet

2024-06-08 14:55:11.595215: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Example Code for Evaluating Our model

This example is made based on subject-dependent paradigm in Physionet dataset.

In [2]:
dataset = np.load('data/Physionet.npy', allow_pickle=True).item()

In [3]:
def split_data(fold, dataset):
    X = [dataset[i]['X'] for i in dataset.keys() if i not in [87, 88, 91, 99, 103]]
    y = [dataset[i]['y'] for i in dataset.keys() if i not in [87, 88, 91, 99, 103]]
    X = np.concatenate(X)
    y = np.concatenate(y)

    n = 0
    kf = KFold(n_splits=5, random_state=2022, shuffle=True)
    for train_index, test_index in kf.split(X):
        if n == fold:
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=2022)
            break
        n = n + 1
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [4]:
MODEL_NAME = 'Ours'
pred = []
gt = []
model = Graph_EEGresnet.irfanet(960, 64, 128, 4, '', 0.075)
for fold in range(5):
    model_path = 'models/%s/%s/weights.hdf5'%(MODEL_NAME, str(fold))
    model.load_weights(model_path)
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(fold, dataset)
    EEG_mov_p = model.predict(X_test)
    pred.append(EEG_mov_p)
    gt.append(y_test)


2024-06-08 14:55:24.562725: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-06-08 14:55:24.595639: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA TITAN Xp computeCapability: 6.1
coreClock: 1.582GHz coreCount: 30 deviceMemorySize: 11.90GiB deviceMemoryBandwidth: 510.07GiB/s
2024-06-08 14:55:24.595676: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-06-08 14:55:24.607623: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-06-08 14:55:24.607673: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-06-08 14:55:24.608998: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.1

In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score,recall_score
EEG_acc = []
EEG_pre = []
EEG_recall = []
matrix = []
for p,g in zip(pred,gt):
    p_0 = np.argmax(p[2], axis=1)
    g = np.argmax(g, axis=1)
    EEG_acc.append(accuracy_score(p_0,g))
    EEG_pre.append(precision_score(p_0,g, average='macro'))
    EEG_recall.append(recall_score(p_0,g,average='macro'))
    matrix.append(confusion_matrix(p_0,g))
np.mean(EEG_acc), np.mean(EEG_pre), np.mean(EEG_recall), EEG_acc

(0.7356108859293572,
 0.7365256327057406,
 0.7379267609583741,
 [0.7353792704111175,
  0.7295888824551245,
  0.7365373480023162,
  0.7353792704111175,
  0.7411696583671106])